<h3>Importing the Libraries

In [1]:
import os
import cv2
import argparse
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import torchvision
import torchvision.transforms as transforms

from efficientnet_pytorch import EfficientNet

In [2]:
for file_name in os.listdir():
    name, extension = os.path.splitext(file_name)
    if extension == '.png':
        original_image = cv2.imread(file_name)
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(original_image, (100, 100))
        cv2.imwrite(file_name, resized_image)

In [3]:
def class_initiator():
    file_array = []
    for file_name in os.listdir():
        name, extension = os.path.splitext(file_name)
        if extension == '.png':
            file_array.append(file_name)
    return file_array

In [4]:
file_array = class_initiator()
class_dictionary = {
            'file_name': file_array
          }
df = pd.DataFrame(class_dictionary)
df.to_csv('dataset.csv')

In [5]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform = None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        img_path = os.path.join(str(self.root_dir), str(self.annotations.iloc[index, 1]))
        image = plt.imread(img_path)
        
        if self.transform:
            image = self.transform(image)
        return (image)

In [6]:
root_directory = os.getcwd()
dataset = CustomDataset(csv_file = "dataset.csv", root_dir = root_directory, transform = transforms.ToTensor())
batch_size = 500
test_loader = DataLoader(dataset = dataset, batch_size = batch_size, shuffle = True)

<h3>Neural Network Class

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 128, 5, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(10368, 4096)
        self.fc2 = nn.Linear(4096, 1024)
        self.fc3 = nn.Linear(1024, 128)
        self.fc4 = nn.Linear(128, 9)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)

        return x

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = (1e-3))

In [9]:
model.load_state_dict(torch.load('0602-674712683-SubramanianShunmugam.pt'))

<All keys matched successfully>

In [10]:
for values in test_loader:
    output = model(values)
    pred = output.argmax(dim=1, keepdim=True)

In [18]:
count = 0
for output_value in pred:
    print("File Name: ", file_array[count])
    if output_value.item() == 0:
        print("Circle")
    if output_value.item() == 1:
        print("Square")
    if output_value.item() == 2:
        print("Octagon")
    if output_value.item() == 3:
        print("Heptagon")
    if output_value.item() == 4:
        print("Nonagon")
    if output_value.item() == 5:
        print("Star")
    if output_value.item() == 6:
        print("Hexagon")
    if output_value.item() == 7:
        print("Pentagon")
    if output_value.item() == 8:
        print("Triangle")
    count+=1

File Name:  Circle_000dfc5c-2a92-11ea-8123-8363a7ec19e6.png
Circle
File Name:  Circle_000ed2d8-2a8a-11ea-8123-8363a7ec19e6.png
Circle
File Name:  Circle_00a80338-2a92-11ea-8123-8363a7ec19e6.png
Circle
File Name:  Circle_00b9c086-2a8e-11ea-8123-8363a7ec19e6.png
Circle
File Name:  Circle_00bb366e-2a89-11ea-8123-8363a7ec19e6.png
Circle
File Name:  Circle_00c64264-2a95-11ea-8123-8363a7ec19e6.png
Circle
File Name:  Circle_00d04b8c-2a91-11ea-8123-8363a7ec19e6.png
Circle
File Name:  Circle_00d9c254-2a99-11ea-8123-8363a7ec19e6.png
Circle
File Name:  Circle_00dee67a-2a86-11ea-8123-8363a7ec19e6.png
Nonagon
File Name:  Circle_00e3ff88-2a96-11ea-8123-8363a7ec19e6.png
Nonagon
File Name:  Circle_00ed5faa-2a88-11ea-8123-8363a7ec19e6.png
Circle
File Name:  Circle_00f3a342-2a97-11ea-8123-8363a7ec19e6.png
Circle
File Name:  Circle_00f5cb02-2a94-11ea-8123-8363a7ec19e6.png
Nonagon
File Name:  Circle_00f6e2b8-2a84-11ea-8123-8363a7ec19e6.png
Circle
File Name:  Circle_0a05eaf4-2a8c-11ea-8123-8363a7ec19e6.png